In [28]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1wDtYhmJuMUaGTNymPmQbiJYhF7MhDoZ4'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')  

downloaded = drive.CreateFile({'id':'1B3Y9ygdKTY3S_OPuZhSNyWRMkIiwt3Q3'}) # replace the id with id of file you want to access
downloaded.GetContentFile('test.csv')  



import numpy as np 
import pandas as pd
import tensorflow as tf

train_db = pd.read_csv("train.csv")
test_db  = pd.read_csv("test.csv")
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint

saved_weights_name='ANNWeights.h5'

num_classes =3
y_train = train_db.loc[:,'class']

print ("y_train:", y_train.shape)

y_train = np_utils.to_categorical(y_train, num_classes)
print ("y_train:", y_train.shape)
print(y_train)
x_train = train_db.loc[:,'B':'R']

print ("x_train:",x_train.shape)

inp = Input(shape=(3,))
hidden_1 = Dense(100, activation='relu')(inp)
dropout_1 = Dropout(0.2)(hidden_1)
out = Dense(num_classes ,activation='softmax')(hidden_1)
model = Model(input=inp, output=out)






checkpoint = ModelCheckpoint(saved_weights_name, 
                                     monitor='val_acc', 
                                     verbose=1, 
                                     save_best_only=True, 
                                     mode='max')

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=500, nb_epoch=1,
          verbose=1, validation_split=0.1,
         callbacks=[checkpoint]) # ...holding out 10% of the data for validation


x_test = test_db.loc[:,'B':'R']

print(np.argmax(model.predict(x_test))) # Evaluate the trained model on the test set!



('y_train:', (145186,))
('y_train:', (145186, 3))
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
('x_train:', (145186, 3))


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:51: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:71: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 130667 samples, validate on 14519 samples
Epoch 1/1
130667/130667 [==============================] - 1s 11us/step - loss: 5.0212 - acc: 0.6886 - val_loss: 3.2821e-04 - val_acc: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to ANNWeights.h5
2
